# sql.py
- 1. 모듈 구성
    + Save_DB : 데이터베이스 생성 및 데이터베이스에 데이터 저장 자동화 코드
    + Load_DB : 데이터베이스에서 데이터 조회 및 판다스 데이터테이블 추출 자동화 코드

- 2. 모듈 사용방법
    + 1. sql.Save_DB(data, weather)
        * data는 사내급식 관련 csv
        * weather는 기상청 날씨 csv
        * db 파일을 안만들었으면 sql.Save_DB(data, weather).make_db() 명령어 입력
        * 데이터 가공 후 데이터베이스에 데이터값들 저장
            + sql.Save_DB(data, weather).save() : sql에 저장하기 위해 데이터 전처리 및 csv로 저장해주는 자동화 코드
        * csv 파일 읽어서 db에 데이터 입력
            + sql.Save_DB(data, weather).input()
    + 2. sql.Load_DB() : DB에 있는 데이터테이블을 판다스 데이터프레임으로 추출하는 자동화 코드

In [2]:
import numpy as np
import pandas as pd
import sqlite3
from os import mkdir
import csv

In [2]:
import sql

# 0. 모듈 호출

In [3]:
data = pd.read_csv("../data/train.csv")
weather = pd.read_csv("../data/weather.csv", encoding='euc-kr')
Save_DB = sql.Save_DB(data, weather)

In [4]:
data

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20,수,2983,75,198,4,391.0,모닝롤/페퍼로니피자 우유/주스 계란후라이/찐계란 크루통크림스프/흑미밥 아귀지리 마늘...,쌀밥/흑미밥/찰현미밥 아욱국 수제함박스테이크 견과류마카로니범벅 생깻잎지 단호박물김치...,김치볶음밥 미니쫄우동*맛살튀김 브로콜리깨소스무침 계란후라이 고들빼기무침 겉절이김치,1093.0,421.0
1201,2021-01-21,목,2983,92,231,462,351.0,모닝롤/생크림단팥빵 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 떡국 해물땡굴소스볶...,쌀밥/수수밥/찰현미밥 냉이된장국 동파육 봄동전 청경채/버섯숙회*초장 무생채 양상추샐...,흑미밥 쇠고기무국 삼치양념구이 비엔나채소볶음 숙주나물당근무침 포기김치,832.0,353.0
1202,2021-01-22,금,2983,255,248,1,303.0,모닝롤/BLT샌드위치 우유/주스 계란후라이/찐계란 흑임자죽/흑미밥 바지락살국 두부조...,전주비빔밥*약고추장 계란파국 요거닭 올방개묵무침 파프리카해초무침 포기김치 양상추샐러...,흑미밥 수제비국 수제맛쵸킹탕수육 유부채소겨자냉채 참나물무침 갓김치/겉절이김치,579.0,217.0
1203,2021-01-25,월,2983,107,153,616,327.0,모닝롤/호박고구마오븐구이 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 감자양파국 분...,쌀밥/흑미밥/찰현미밥 전주식콩나물해장국 돈육간장불고기 깐풍연근 연두부*달래양념장 봄...,흑미밥 열무된장국 장어강정*데리야끼소스 깻잎쌈*생강채 오이선 포기김치,1145.0,502.0


In [5]:
weather

,지점,지점명,일시,평균기온(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%)
0,192,진주,2016-02-01,-0.6,NaN,1.3,43.9
1,192,진주,2016-02-02,-2.3,NaN,0.8,47.5
2,192,진주,2016-02-03,-1.7,NaN,0.6,57.1
3,192,진주,2016-02-04,-0.2,NaN,0.7,53.4
4,192,진주,2016-02-05,1.3,NaN,1.0,44.4
...,...,...,...,...,...,...,...
2171,192,진주,2022-01-11,0.1,NaN,2.3,44.4
2172,192,진주,2022-01-12,-2.2,NaN,1.1,42.0
2173,192,진주,2022-01-13,-2.4,NaN,2.0,43.8
2174,192,진주,2022-01-14,-3.0,NaN,0.5,47.5


# 1. DB 생성
- OK

In [6]:
# sqlite3로 db 파일 생성 / db 내 테이블 만들기 함수
# 폴더 내 data/team3.db가 생성
Save_DB.make_db()

# 2. 데이터 전처리
- 1. make_csv()
- 2. split()
    + 두 함수 모두 save() 함수 호출시에 자동 작동됩니다. 따라서, 아래 코드는 잘 작동되는지 확인하기 위해 작성했습니다.
    + 두 함수 모두 문제없이 진행된다는 것 확인할 수 있습니다.

In [5]:
# 실제 코드에선 make_csv()함수 사용하지 않습니다. save() 함수 추출시 자동으로 작동되도록 모듈 설계했습니다.
# 아래 코드는 단순히 make.csv() 함수가 잘 작동되는지 확인하기 위해 입력했습니다.
# 만약 make_csv() 함수 시행하고 split() 시행하면 전처리 코드가 두번 진행되어 오류 발생합니다.
# prepro = Save_DB.make_csv()

/Users/hoon/Desktop/Data/vscode/team_project/Forecast_final/module/sql.py:263: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['dinner_rice'][index] = '밥'
/Users/hoon/Desktop/Data/vscode/team_project/Forecast_final/module/sql.py:265: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['dinner_rice'][index] = '밥'


In [6]:
# prepro

,datetime,weekdays,worker_number,real_number,vacation_number,biztrip_number,overtime_number,telecom_number,lunch_number,dinner_number,...,lunch_main,dinner_rice,dinner_soup,dinner_main,temperature,rain,wind,humidity,discomfort_index,perceived_temperature
0,2016-02-01,monday,2601,2401.0,50,150,238,0.0,1039.0,331.0,...,쇠불고기,밥,육개장,자반고등어구이,-0.6,0.0,1.3,43.9,39.275534,0.641548
1,2016-02-02,tuesday,2601,2378.0,50,173,319,0.0,867.0,560.0,...,가자미튀김,콩나물밥*양념장,어묵국,유산슬,-2.3,0.0,0.8,47.5,36.562925,-0.160634
2,2016-02-03,wednesday,2601,2365.0,56,180,111,0.0,1017.0,573.0,...,치킨핑거,밥,청국장찌개,황태양념구이,-1.7,0.0,0.6,57.1,35.796707,0.964631
3,2016-02-04,thursday,2601,2277.0,104,220,355,0.0,978.0,525.0,...,주꾸미볶음,미니김밥*겨자장,우동,멕시칸샐러드,-0.2,0.0,0.7,53.4,38.396068,2.181494
4,2016-02-05,friday,2601,2142.0,278,181,34,0.0,925.0,330.0,...,돈육씨앗강정,밥,차돌박이찌개,닭갈비,1.3,0.0,1.0,44.4,41.575228,3.073400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20,wednesday,2983,2319.0,75,198,4,391.0,1093.0,421.0,...,수제함박스테이크,김치볶음밥,미니쫄우동*맛살튀김,브로콜리깨소스무침,-0.4,0.0,0.5,64.0,36.570560,2.553014
1201,2021-01-21,thursday,2983,2309.0,92,231,462,351.0,832.0,353.0,...,동파육,밥,쇠고기무국,삼치양념구이,5.2,1.7,0.6,77.8,43.391744,7.774119
1202,2021-01-22,friday,2983,2177.0,255,248,1,303.0,579.0,217.0,...,요거닭,밥,수제비국,수제맛쵸킹탕수육,7.0,1.2,0.5,97.5,44.784250,9.778207
1203,2021-01-25,monday,2983,2396.0,107,153,616,327.0,1145.0,502.0,...,돈육간장불고기,밥,열무된장국,장어강정*데리야끼소스,6.3,0.0,1.0,75.8,45.291246,8.163400


In [6]:
hr, lunch, dinner, weather, calender = Save_DB.split()

/Users/hoon/Desktop/Data/vscode/team_project/Forecast_final/module/sql.py:263: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['dinner_rice'][index] = '밥'
/Users/hoon/Desktop/Data/vscode/team_project/Forecast_final/module/sql.py:265: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['dinner_rice'][index] = '밥'


In [7]:
hr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1205 entries, 0 to 1204
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   datetime         1205 non-null   datetime64[ns]
 1   worker_number    1205 non-null   int64         
 2   real_number      1205 non-null   float64       
 3   biztrip_number   1205 non-null   int64         
 4   overtime_number  1205 non-null   int64         
 5   telecom_number   1205 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 65.9 KB


In [8]:
lunch.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1205 entries, 0 to 1204
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   datetime      1205 non-null   datetime64[ns]
 1   new_lunch     1205 non-null   object        
 2   lunch_rice    1205 non-null   object        
 3   lunch_soup    1205 non-null   object        
 4   lunch_main    1205 non-null   object        
 5   lunch_number  1205 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 65.9+ KB


In [9]:
dinner.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1205 entries, 0 to 1204
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   datetime       1205 non-null   datetime64[ns]
 1   new_dinner     1205 non-null   object        
 2   dinner_rice    1205 non-null   object        
 3   dinner_soup    1205 non-null   object        
 4   dinner_main    1205 non-null   object        
 5   dinner_number  1205 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 65.9+ KB


In [10]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1205 entries, 0 to 1204
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   datetime               1205 non-null   datetime64[ns]
 1   temperature            1204 non-null   float64       
 2   rain                   1205 non-null   float64       
 3   wind                   1204 non-null   float64       
 4   humidity               1205 non-null   float64       
 5   discomfort_index       1204 non-null   float64       
 6   perceived_temperature  1203 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 75.3 KB


In [11]:
calender.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1205 entries, 0 to 1204
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   datetime  1205 non-null   datetime64[ns]
 1   year      1205 non-null   object        
 2   month     1205 non-null   object        
 3   date      1205 non-null   object        
 4   weekdays  1205 non-null   object        
 5   season    1205 non-null   object        
 6   vacation  1205 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 75.3+ KB


# 3. DB 저장
- 1. data 폴더에 csv 파일 생성 확인
- 2. 가능하면 sqlite로 db에 잘 저장되었는지 확인
- 3. 중복 입력을 막기 위해  datetime 필드는 key-value로 설정해주자

### 1. save
- sql.Save_DB(data, weather).save()
- data 폴더에 데이터 csv파일 잘 저장되는지 확인

In [5]:
Save_DB.save()

/Users/hoon/Desktop/Data/vscode/team_project/Forecast_final/module/sql.py:264: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['dinner_rice'][index] = '밥'
/Users/hoon/Desktop/Data/vscode/team_project/Forecast_final/module/sql.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['dinner_rice'][index] = '밥'


### 2. input
- sql.Save_DB(data, weather).input()
- 이 함수들은 경로 지정이 중요
- 에러 발생시 sql.py에서 저장경로와 csv 파일을 읽는 경로를 확인

In [7]:
Save_DB.input()

# 4. Load_DB
- DB에서 데이터 추출해 판다스 데이터프레임으로 파이썬 위에 올려두기

In [8]:
df = sql.Load_DB()
df

,datetime,weekdays,worker_number,real_number,vacation_number,biztrip_number,overtime_number,telecom_number,lunch_number,dinner_number,...,lunch_main,dinner_rice,dinner_soup,dinner_main,temperature,rain,wind,humidity,discomfort_index,perceived_temperature
0,2016-02-01,monday,2601,2401,50,150,238,0,1039,331,...,쇠불고기,밥,육개장,자반고등어구이,-0.6,0.0,1.3,43.9,39.275534,0.641548
1,2016-02-02,tuesday,2601,2378,50,173,319,0,867,560,...,가자미튀김,콩나물밥*양념장,어묵국,유산슬,-2.3,0.0,0.8,47.5,36.562925,-0.160634
2,2016-02-03,wednesday,2601,2365,56,180,111,0,1017,573,...,치킨핑거,밥,청국장찌개,황태양념구이,-1.7,0.0,0.6,57.1,35.796707,0.964631
3,2016-02-04,thursday,2601,2277,104,220,355,0,978,525,...,주꾸미볶음,미니김밥*겨자장,우동,멕시칸샐러드,-0.2,0.0,0.7,53.4,38.396068,2.181494
4,2016-02-05,friday,2601,2142,278,181,34,0,925,330,...,돈육씨앗강정,밥,차돌박이찌개,닭갈비,1.3,0.0,1.0,44.4,41.575228,3.0734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20,wednesday,2983,2319,75,198,4,391,1093,421,...,수제함박스테이크,김치볶음밥,미니쫄우동*맛살튀김,브로콜리깨소스무침,-0.4,0.0,0.5,64.0,36.57056,2.553014
1201,2021-01-21,thursday,2983,2309,92,231,462,351,832,353,...,동파육,밥,쇠고기무국,삼치양념구이,5.2,1.7,0.6,77.8,43.391744,7.774119
1202,2021-01-22,friday,2983,2177,255,248,1,303,579,217,...,요거닭,밥,수제비국,수제맛쵸킹탕수육,7.0,1.2,0.5,97.5,44.78425,9.778207
1203,2021-01-25,monday,2983,2396,107,153,616,327,1145,502,...,돈육간장불고기,밥,열무된장국,장어강정*데리야끼소스,6.3,0.0,1.0,75.8,45.291246,8.1634


# 5. sql.py 모듈 진행 코드
- 식수인원 프로젝트 sql 관련 모듈은 다음과 같이 진행하면 됩니다.

In [4]:
import numpy as np
import pandas as pd
import sqlite3
from os import mkdir
import csv
import sql

data = pd.read_csv("../data/train.csv")
weather = pd.read_csv("../data/weather.csv", encoding='euc-kr')
Save_DB = sql.Save_DB(data, weather)

### 1. DB 만들기. DB 있으면 생략
Save_DB.make_db()

### 2. SQL에 맞게 데이터프레임 조작
Save_DB.save()
Save_DB.input()

### 3. SQLite3로 DB에서 데이터프레임 추출
df = sql.Load_DB()
df



/Users/hoon/Desktop/Data/vscode/team_project/Forecast_final/module/sql.py:264: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['dinner_rice'][index] = '밥'
/Users/hoon/Desktop/Data/vscode/team_project/Forecast_final/module/sql.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['dinner_rice'][index] = '밥'


,datetime,weekdays,worker_number,real_number,vacation_number,biztrip_number,overtime_number,telecom_number,lunch_number,dinner_number,...,lunch_main,dinner_rice,dinner_soup,dinner_main,temperature,rain,wind,humidity,discomfort_index,perceived_temperature
0,2016-02-01,monday,2601,2401,50,150,238,0,1039,331,...,쇠불고기,밥,육개장,자반고등어구이,-0.6,0.0,1.3,43.9,39.275534,0.641548
1,2016-02-02,tuesday,2601,2378,50,173,319,0,867,560,...,가자미튀김,콩나물밥*양념장,어묵국,유산슬,-2.3,0.0,0.8,47.5,36.562925,-0.160634
2,2016-02-03,wednesday,2601,2365,56,180,111,0,1017,573,...,치킨핑거,밥,청국장찌개,황태양념구이,-1.7,0.0,0.6,57.1,35.796707,0.964631
3,2016-02-04,thursday,2601,2277,104,220,355,0,978,525,...,주꾸미볶음,미니김밥*겨자장,우동,멕시칸샐러드,-0.2,0.0,0.7,53.4,38.396068,2.181494
4,2016-02-05,friday,2601,2142,278,181,34,0,925,330,...,돈육씨앗강정,밥,차돌박이찌개,닭갈비,1.3,0.0,1.0,44.4,41.575228,3.0734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20,wednesday,2983,2319,75,198,4,391,1093,421,...,수제함박스테이크,김치볶음밥,미니쫄우동*맛살튀김,브로콜리깨소스무침,-0.4,0.0,0.5,64.0,36.57056,2.553014
1201,2021-01-21,thursday,2983,2309,92,231,462,351,832,353,...,동파육,밥,쇠고기무국,삼치양념구이,5.2,1.7,0.6,77.8,43.391744,7.774119
1202,2021-01-22,friday,2983,2177,255,248,1,303,579,217,...,요거닭,밥,수제비국,수제맛쵸킹탕수육,7.0,1.2,0.5,97.5,44.78425,9.778207
1203,2021-01-25,monday,2983,2396,107,153,616,327,1145,502,...,돈육간장불고기,밥,열무된장국,장어강정*데리야끼소스,6.3,0.0,1.0,75.8,45.291246,8.1634
